In [20]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import random
import datetime


In [10]:
def get_header(fn='../data/headers.json', index=-1):
    with open(fn, 'r') as fid:
        obj = json.load(fid)
        if 0 <= index < len(obj):
            return obj[index]
        else:
            return random.sample(obj, 1)[0]

def get_option_from_yf(symbol='KRE', date='1737072000', output_dir='.'):
    fnout = f'{output_dir}/html_{symbol}_{date}.txt'
    if os.path.exists(fnout):
        with open(fnout, 'r') as fid:
            return fid.read()
    else:
        url = f'https://finance.yahoo.com/quote/{symbol}/options/?date={date}'
        res = requests.get(url, headers=HEADER)
        if res.status_code == 200:
            with open(fnout, 'w') as fid:
                fid.write(res.text)
            return res.text
        else:
            raise ValueError('Failed in connectting YF: res.status_code =', res.status_code)


def parse_table(div):
    table = defaultdict(list)
    for i, tr in enumerate(div.findAll('tr')):
        if i == 0:
            columns = [td.text.strip() for td in tr.findAll('th')]
        else:
            for j, td in enumerate(tr.findAll('td')):
                print(j, td.text.strip())
                table[columns[j]].append(td.text.strip())
    return table


def parse_html(text):
    soup = BeautifulSoup(text, 'lxml')
    obj = dict()
    h1 = soup.find('section', class_='paddingRight')
    if h1:
        obj['long_name'] = h1.text.strip()
    tmp = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'})
    if tmp:
        obj['live_price'] = tmp.text.strip()
    tables = soup.find('section', {'data-testid': 'options-list-table'}).findAll('div', class_='tableContainer')
    obj['call'] = parse_table(tables[0])
    obj['put'] = parse_table(tables[1])
    return obj

In [11]:
url = 'https://finance.yahoo.com/quote/BSX/options/'
res = requests.get(url, headers=get_header())
res.status_code

200

In [19]:
soup = BeautifulSoup(res.text, 'html')
tmp = soup.find('div', {'data-testid': 'options-toolbar'})
tmp.find

' Aug 16, 2024   Aug 16, 2024  Sep 20, 2024  Nov 15, 2024  Jan 17, 2025  Feb 21, 2025  Jun 20, 2025  Jan 16, 2026  Jun 18, 2026  Dec 18, 2026     All Strike Prices   All Strike Prices  37.50  40.00  42.50  45.00  47.50  50.00  52.50  55.00  57.50  60.00  62.50  65.00  67.50  70.00  72.50  75.00  77.50  80.00  82.50  85.00  90.00  95.00  100.00     List   List  Straddle     All Options   All Options  Calls  Puts    '

In [23]:
datetime.datetime(1970, 1, 1) + datetime.timedelta(seconds=1723766400)

datetime.datetime(2024, 8, 16, 0, 0)

In [41]:
for d in tmp.find('div', {'role': 'listbox'}).find_all('div', {'role': 'option'}):
    print(pd.to_datetime(d.text))
    #print(datetime.datetime(d.text.strip()))


2024-08-16 00:00:00
2024-09-20 00:00:00
2024-11-15 00:00:00
2025-01-17 00:00:00
2025-02-21 00:00:00
2025-06-20 00:00:00
2026-01-16 00:00:00
2026-06-18 00:00:00
2026-12-18 00:00:00
